<img src='../img/cover_lander.png'/>


# Taxi Autónomo (SmartCab)

## Objetivo

El trabajo de un SmartCab es recoger al pasajero en un lugar y dejarlo en otro. Algunos detalles que nos encantaría que nuestro Smartcab tenga en cuenta serían:

* Dejar al pasajero en la ubicación correcta.  
* Ahorrar tiempo al pasajero dedicando el mínimo tiempo posible para dejarlo.  
* Cuidar la seguridad del pasajero y respetar las normas de tráfico.

En este Notebook se va a evaluar el aprendizaje del taxi autónomo sin refuerzo (movimientos aleatorios), frente al aprendizaje por refurezo mediante Q-Learning.

## Recompensas

* El agente debería recibir una alta recompensa positiva por una entrega del cliente exitosa porque este comportamiento es de los más importantes que queremos que aprenda.
* El agente debería ser penalizado si intenta dejar a un pasajero en destinos incorrectos.
* El agente debería recibir una ligera recompensa negativa por no llegar a destino después de cada intervalo de tiempo. "Ligera" negativa porque preferiríamos que nuestro agente llegue tarde en lugar de hacer movimientos erróneos tratando de llegar al destino lo más rápido posible."

Para este ejercicio vamos a establecer las siguientes "recompensas":
* Recibimos +20 puntos por un traslado exitoso.  
* Perdemos 1 punto por cada intervalo de tiempo que tarda.  
* También hay una penalización de 10 puntos por acciones de recogida y dejada ilegales. 
* Un movimiento que de lugar a un "choque" tiene la penalización de perder tiempo (el taxi no se desplaza pero se recibe la penalización de duración)

## Espacio de estados

El espacio de estados es el conjunto de todas las posibles situaciones en las que nuestro taxi podría estar. El estado además debe contener información útil que el agente necesite para tomar la acción correcta.

<img src='../img/smartcab_map.png' width='350' />

Supongamos que Smartcab es el único vehículo en este circuito de aprendizaje. Nuestro circuito está dividido en __una cuadrícula de 5x5__, lo que nos da __25 posibles ubicaciones__ para el taxi (posiciones (0,0) a (5,5)). Estas 25 ubicaciones __son una parte de nuestro espacio de estados__. Se puede observar que la __ubicación actual__ de nuestro taxi es la __coordenada (3, 1)__.

También se puede ver que hay __cuatro (4) ubicaciones__ en las que podemos __recoger y dejar__ a un pasajero: R, G, Y, B o [(0,0), (0,4), (4,0), (4,3)] en coordenadas (fila, columna).  También debemos tener en cuenta un (1) estado adicional del pasajero de estar dentro del taxi.

Por tanto, hay un total de 5 x 5 x 5 x 4 = 500 estados.

## Espacio de acciones

El agente se encuentra con uno de los 500 estados y toma una acción. La acción en nuestro caso puede ser moverse en una dirección o decidir recoger/dejar a un pasajero. En otras palabras, tenemos seis posibles acciones:

0. sur
1. norte
2. este
3. oeste
4. recoger
5. dejar

Según la ilustración, el taxi no puede realizar ciertas acciones en ciertos estados debido a las paredes (por ejemplo mover de la posición (3,1) a la (3,0)).  El entorno proporciona una penalización de -1 por cada movimiento no permitido y el taxi no se moverá a ningún lado. Como siguiente estado devuelve el mismo de partida.



## Implementación

In [1]:
import gymnasium as gym
import warnings

In [2]:
# Saltarnos la limitación de 200
env = gym.make('Taxi-v3', render_mode='ansi').env

state, info = env.reset(seed=19)
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




* El cuadrado relleno representa el taxi, que es de color amarillo sin un pasajero y verde con un pasajero. En este caso, comenzaríamos con nuestro taxi situado en (2,0).
* La barra ("|") representa una pared que el taxi no puede cruzar.
* R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra azul representa la ubicación actual de recogida del pasajero, y la letra morada es el destino actual. Es decir hay que recogerlo de Y y entregarlo en R, para este ejemplo generado al resetear el entorno con la semilla ajustada al valor 19.

In [3]:
print('Estado inicial:', state)
print('Info extra:', info)

Estado inicial: 208
Info extra: {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)}


In [4]:
# Desenvuelve el entorno base y llama al método decode en el entorno base
list(env.unwrapped.decode(208))

[2, 0, 2, 0]

Si quisieramos cambiar la posición de partida de nuestro taxi y llevarlo a la de la figura, podemos moverlo con las acciones e ignorar las recompensas. Bastaría con moverlo al este y luego al sur.

In [5]:
# Mover el taxi al este (2) y luego al sur (0)
movements = [2, 0]

for action in movements:
    env.step(action)
    print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)



## Método 1: Aprendizaje sin refuerzo. Movimientos aleatorios

In [6]:
# Entorno con movimientos aleatorios
timesteps = 0
penalties, rewards = 0, 0
simulated_time = 0
frames = []

done = False
while not done:
    action = env.action_space.sample()
    if action < 4:
        simulated_time += 45
    else:
        simulated_time += 75
    state, reward, done, truncated, info = env.step(action=action)
    if reward == -10:
        penalties += 1
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward,
        'elapsed': simulated_time
    })
    timesteps += 1
    
print('Pasos de tiempo:', timesteps)
print('Penalizaciones:', penalties)
print('Tiempo real simulado:', simulated_time)

Pasos de tiempo: 1444
Penalizaciones: 460
Tiempo real simulado: 79380


En tiempo de ejecución de código no ha sido nada. Pero si imaginamos que hubiera sido una prueba en un entorno real el resultado no sería aceptable. Esta es una de las razones por las que se usan entornos simulados para aprender. Aunque no siempre será posible o recomendable (por ejemplo, los coches autónomos de verdad aprenden sobre el terreno).

### Visualización

Para poder ver qué es lo que realmente está haciendo el taxi autónomo, hemos creado una función que pinta cada frame, dejando un intervalo de tiempo entre frame y frame para crear el efecto de animación.

In [7]:
import os
import sys

# add "src" path
root_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if os.path.exists(root_path) and root_path not in sys.path: 
    sys.path.append(root_path)

from utils.functions import episode_animation

In [8]:
# Lanzamos la animacion, pero asegurándonos de que sólo mostramos como mucho 500 frames.
# Si se desea ver completa sólo hay que quitar el indexado y volver a ejecutar
episode_animation(frames[max(0,len(frames)-500):]) 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 500
State: 0
Action: 5
Reward: 20
Elapsed time (sec.): 79380


Cómo se puede comprobar este mecanismo, aunque cumple con el cometido, tiene severas limitaciones:
- Los tiempos son inaceptables.
- No se guarda el "aprendizaje", pero aunque lo hiciéramos, sólo valdría para la situación de partida (el taxi en la posición 3,1. Recoger en "Y", y entregar en "R")

Necesitamos un mecanismo de mejora de la estrategia que no sólo reduzca los tiempos y los optimice al máximo, sino que además sea flexible como para poder aplicarse en cualquier situación. Ese es el objetivo del Q-Learning.

## Método 2: Aprendizaje por refuerzo

Reiniciamos el entorno para ello.

In [9]:
# Reiniciar el entorno
env = gym.make("Taxi-v3", render_mode = "ansi").env
state, info = env.reset(seed=19)

# Renderizar el entorno
print(env.render())

# Imprimir el estado actual (devuelto por reset)
print("Current State:", state)

# Imprimir el espacio de acciones
print("Action Space: {}".format(env.action_space))

# Imprimir el espacio de observaciones
print("State Space: {}".format(env.observation_space))


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Current State: 208
Action Space: Discrete(6)
State Space: Discrete(500)


In [10]:
# Movimiento al Este y al Sur
movements = [2, 0]

for mov in movements:
    # Realizar la acción
    state, reward, done, truncated, info = env.step(mov)
    
    print(env.render())             # Mostrar el entorno
    print("Estado:", state)         # Mostrar el estado actual
    print("Recompensa:", reward)    # Mostrar la recompensa obtenida
    print("Terminó:", done)         # Mostrar si el episodio terminó
    print("Truncado:", truncated)   # Mostrar si el episodio fue truncado

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Estado: 228
Recompensa: -1
Terminó: False
Truncado: False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

Estado: 328
Recompensa: -1
Terminó: False
Truncado: False


### Q-Learning

Los pasos del algoritmo de Q-Learning epsilo-greedy que nos permitirá estimar la Q-table son los siguientes:

* Inicializar la tabla Q con todos ceros.
* Seleccionar los valores de los hiperparámetros
* Comenzar a explorar acciones: Para cada estado, seleccione cualquiera entre todas las acciones posibles para el estado actual (S).
* Viajar al siguiente estado (S') como resultado de esa acción (a).
* Para todas las acciones posibles desde el estado (S') seleccione la que tenga el valor Q más alto.
* Actualizar los valores de la tabla Q usando la ecuación ya vista.
* Establecer el siguiente estado como el estado actual.
* Si se alcanza el estado objetivo, entonces terminar y repetir el proceso.

Lo primero creamos la estructura de datos que nos permita almacenar la Q-table

In [11]:
import random
import time
from random import sample
from time import sleep

import numpy as np
from IPython.display import clear_output

In [12]:
np.__version__

'2.2.1'

In [13]:
# Inicializar tabla-Q 
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], shape=(500, 6))

In [14]:
print(q_table.shape)
print(q_table.size)

(500, 6)
3000


Lo siguiente es seleccionar los valores de los hiperparámetros, alpha, gamma y épsilon.

In [15]:
alpha = 0.05
gamma = 0.9
epsilon = 0.1


Ahora podemos crear el algoritmo de entrenamiento que actualizará esta tabla Q a medida que el agente explore el entorno a lo largo de miles de episodios.




In [16]:
# Para ver el tiempo de CPU
# %%time

all_epochs = []
all_penalties = []
num_episodes = 100_000

init_time = time.time()

for i in range(1, num_episodes + 1):
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, truncated, info = env.step(action)
        
        next_max = np.max(q_table[next_state])   # maxQ(s', a')
        old_value = q_table[state, int(action)]
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        
        q_table[state, int(action)] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f'Episodio: {i}, {i/num_episodes * 100:.2f}')
    
    state, info = env.reset()

print('Entrenamiento finalizado')
print(f'Tiempo de entrenamiento: {round(time.time() - init_time, 1)} segundos')
            

Episodio: 100000, 100.00
Entrenamiento finalizado
Tiempo de entrenamiento: 20.1 segundos


### Evaluación

Vamos a evaluar el rendimiento de nuestro agente. Ya no necesitamos explorar acciones, así que ahora la siguiente acción siempre se selecciona usando el mejor valor Q.


In [17]:
# Uso de la tabla-Q
total_epochs, total_penalties, total_rewards = 0, 0, 0
num_episodes = 100

state, info = env.reset(seed=19)

# Tendrá un elemento por episodio que contendrá los 
# frames de ese episodio e información adicional
set_frames = []

for i in range(1, num_episodes + 1): 
    epochs, penalties, reward = 0, 0, 0
    done = False
    frames = []
    
    while not done: 
        action = np.argmax(q_table[state])
        state, reward, done, truncated, info = env.step(action)
        
        total_rewards += reward
        frames.append({
            'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward,
            'elapsed': 0
        })
        
        if reward == -10:
            penalties += 1
            total_penalties += 1
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait = True)
        print(f"Episodio: {i}, {i/num_episodes * 100:.2f} %")
        
    set_frames.append(frames)
    total_epochs += epochs
    total_penalties += penalties
    state, info = env.reset()
    
print(f'Resultados después de {num_episodes} episodios:')
print(f'Número medio de acciones por episodio: {total_epochs / num_episodes}')
print(f'Número medio de penalizaciones por episodio: {total_penalties / num_episodes}')
print(f'Recompensa media por episodio: {total_rewards / num_episodes}')


Episodio: 100, 100.00 %
Resultados después de 100 episodios:
Número medio de acciones por episodio: 13.22
Número medio de penalizaciones por episodio: 0.0
Recompensa media por episodio: 7.78


Los valores parecen bastante buenos, 13 (aproximadamente) pasos de duración media y no hay penalizaciones (no se intenta recoger o dejar al pasajero en sitios equivocados).

Utilicemos ahora la visualización para ver cuanto de bien ha aprendido a conducir. Vamos a analizar 5 episodios escogidos aleatoriamente.

In [18]:
for frame in sample(set_frames, 5):
    episode_animation(frame[0:1])
    sleep(2)
    episode_animation(frame[1:])
    sleep(1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 17
State: 0
Action: 5
Reward: 20
Elapsed time (sec.): 0


## Comparando nuestro agente de Q-learning con no usar aprendizaje por refuerzo

Vamos a evaluar a nuestros agentes de acuerdo con las siguientes métricas,

* Número promedio de penalizaciones por episodio: Cuanto menor sea el número, mejor será el rendimiento de nuestro agente. Idealmente, nos gustaría que esta métrica sea cero o muy cercana a cero.
* Número promedio de pasos por episodio: También queremos que sea un valor pequeño, que nuestro agente tome la ruta más corta para llegar al destino.
* Recompensas promedio por movimiento: Una recompensa más grande significa que el agente está haciendo lo correcto. Es por eso que decidir las recompensas es una parte crucial del Aprendizaje por Refuerzo.

Recuperemos el código que ya desarrollamos en el método 1 (sin aprendizaje por refuerzo) para obtener los valores anteriores para este escenario y hacer la comparativa.

In [19]:
# Evaluar el comportamiento del agente sin Q-learning
total_epochs, total_penalties, total_rewards = 0, 0, 0
episodes = 100

for _ in range(episodes):
    env.reset()
    
    # Crear el estado inicial
    # state = env.encode(3, 1, 2, 0)
    # env.s = state
    env.s = (3, 1, 2, 0)
    
    # Inicializar las épocas, penalizaciones y recompensas
    epochs, penalties, reward = 0, 0, 0

    done = False
    actions = []
    while not done:
        # Elegir la acción random
        action = env.action_space.sample()
        actions.append(action)
        # Ejecutar la accion
        state, reward, done, truncated, info = env.step(action)
        total_rewards += reward
        # Actualiza el valor de penalizaciones si el reward es -10
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f'Resultados después de {episodes} episodios:')
print(f'Número medio de acciones por episodio: {total_epochs / episodes}')
print(f'Número medio de penalizaciones por episodio: {total_penalties / episodes}')
print(f'Recompensa media por episodio: {total_rewards / episodes}')

Resultados después de 100 episodios:
Número medio de acciones por episodio: 2564.85
Número medio de penalizaciones por episodio: 830.44
Recompensa media por episodio: -10017.81


| **Medida** | **Rendimiento Agente Aleatorio** | **Rendimiento Agente Q-Learning** |
|---------|---------|---------|
| Número medio de acciones por episodio | 2564.85 | 13.22 |
| Número medio de penalizaciones por episodio | 830.44 | 0.0 |
| Recompensa media por episodio | -10017.81 | 7.78 |

Como se puede obsrvar, los resultados del Q-Learning son claramente muy superiores.